### Define and Load Data
- you could start all the analysis with this cell!

In [127]:
import numpy as np
import sys
from pathlib import Path
sys.path.append(r'D:\Neural-Pipeline\source')
from analysis_utils.NeuralDataLoader import NeuralDataLoader, Dots3DMPConfig

subject = "zarya"
date = "20250710"

# Load session
loader = NeuralDataLoader()
loader.load_session(subject, date)

# Load all spike data for different alignments
stimOn_spikes = loader.get_spike_data(alignment='stimOn', good_units_only=True, good_trials_only=True)
saccOnset_spikes = loader.get_spike_data(alignment='saccOnset', good_units_only=True, good_trials_only=True)
postTargHold_spikes = loader.get_spike_data(alignment='postTargHold', good_units_only=True, good_trials_only=True)

# Load tuning data
tuning_spikes = loader.get_tuning_data(good_units_only=True, good_trials_only=True)

# Load behavioral data
behavior_dots3DMP = loader.get_behavioral_data(task='dots3DMP', good_trials_only=True)
behavior_tuning = loader.get_behavioral_data(task='tuning', good_trials_only=True)

# Load Unit Info
unit_info = loader.get_unit_info(good_units_only=True)

# Load configuration
config = Dots3DMPConfig(subject)

# Convert behavioral data
behavior_converted = config.convert_behavioral_data(behavior_dots3DMP, task='dots3DMP')
behavior_tuning_converted = config.convert_behavioral_data(behavior_tuning, task='tuning')

# Get time axes
time_stimOn = config.get_time_axis('stimOn', 'dots3DMP')
time_saccOnset = config.get_time_axis('saccOnset', 'dots3DMP')
time_postTargHold = config.get_time_axis('postTargHold', 'dots3DMP')
time_tuning = config.get_time_axis('stimOn', 'tuning')

Loaded dots3DMP data: zarya20250710dots3DMP_processed.npz
Loaded dots3DMPtuning data: zarya20250710dots3DMPtuning_processed.npz
Loaded dots3DMP configuration for subject: zarya


In [128]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from matplotlib.backends.backend_pdf import PdfPages
import os

depths = unit_info['depth']
real_depth = depths * (-1)

n_time_bins = tuning_spikes.shape[2]
trial_start = config.tuning_time_info['trial_start'] 
bin_size = config.tuning_time_info['bin_size']       
trial_stop = config.tuning_time_info['trial_stop']   
time_axis = trial_start + np.arange(n_time_bins) * bin_size

modality = behavior_tuning['modality']
ves_idx = np.where(modality == 1)[0]
vis_idx = np.where(modality == 2)[0]
com_idx = np.where(modality == 3)[0]
n_mod = 3

n_units, n_trials, n_time_bins = tuning_spikes.shape

depth_bin_size = 20
max_depth = int(np.max(depths))
depth_bins = np.arange(0, max_depth + depth_bin_size, depth_bin_size)
n_depth_bins = len(depth_bins)
slots_per_bin = 5

depth_sort_indices = np.argsort(real_depth, axis=0).flatten()
sorted_depths = depths[depth_sort_indices].flatten()

total_rows = n_depth_bins * slots_per_bin
z_sorted_spkrate = np.full((total_rows, n_time_bins, n_mod), np.nan)

depth_bin_to_base_row = {}
for i, depth_bin in enumerate(depth_bins):
    base_row = (n_depth_bins - 1 - i) * slots_per_bin
    depth_bin_to_base_row[depth_bin] = base_row

depth_bin_slot_counter = {depth_bin: 0 for depth_bin in depth_bins}

for i, unit_idx in enumerate(depth_sort_indices):
    unit_depth = sorted_depths[i]
    bin_idx = int(unit_depth // depth_bin_size) * depth_bin_size
    
    if bin_idx not in depth_bin_to_base_row:
        continue
    
    if depth_bin_slot_counter[bin_idx] >= slots_per_bin:
        continue
    
    unit_data = np.squeeze(tuning_spikes[unit_idx, :, :])
    baseline_mask = time_axis < 0
    
    mean_baseline = np.mean(unit_data[:, baseline_mask])
    std_baseline = np.std(unit_data[:, baseline_mask])
    if std_baseline == 0:
        std_baseline = 1e-6
    if mean_baseline == 0:
        mean_baseline = 1e-6
    unit_data_z = (unit_data - mean_baseline) / std_baseline
    
    base_row = depth_bin_to_base_row[bin_idx]
    slot = depth_bin_slot_counter[bin_idx]
    row_idx = base_row + slot
    
    for mod in range(n_mod):
        if mod == 0:
            unit_data_mod = unit_data_z[ves_idx, :]
        elif mod == 1:
            unit_data_mod = unit_data_z[vis_idx, :]
        else:
            unit_data_mod = unit_data_z[com_idx, :]
        
        mean_response_mod = np.mean(unit_data_mod, axis=0)
        z_sorted_spkrate[row_idx, :, mod] = mean_response_mod
    
    depth_bin_slot_counter[bin_idx] += 1

# Ensure the directory exists
output_dir = r"D:\Neural-Pipeline\results"
os.makedirs(output_dir, exist_ok=True)

pdf_filename = os.path.join(output_dir, f"{date}_dots3DMP_singleneuron_heatmap.pdf")
with PdfPages(pdf_filename) as pdf:
    for mod in range(n_mod):
        plt.figure(figsize=(12, 10))
        
        sns.heatmap(
            z_sorted_spkrate[:, :, mod],
            cmap='RdBu_r',
            center=0,
            vmin=-1, vmax=1,
            cbar_kws={'label': 'Z-scored Firing Rate'},
            xticklabels=False,
            yticklabels=False)

        plt.axvline(x=np.where(time_axis >= 0)[0][0], color='red', linestyle='--')
        
        time_max = time_axis[-1]
        x_pos = np.where(time_axis >= (time_max - trial_stop))[0][0]
        plt.axvline(x=x_pos, color='blue', linestyle='--')
        mod_name = ['Vestibular', 'Visual', 'Combined']
        plt.title(f'Spike Rate Heatmap (ΔF/F₀) - {mod_name[mod]}', fontsize=14, fontweight='bold')

        plt.xlabel('Time (s)', fontsize=12)
        plt.ylabel('Depth (μm)', fontsize=12)

        n_ticks = 6
        tick_indices = np.linspace(0, n_time_bins-1, n_ticks, dtype=int)
        tick_labels = [f'{time_axis[i]:.2f}' for i in tick_indices]
        plt.xticks(tick_indices, tick_labels)

        y_tick_indices = []
        y_tick_labels = []
        
        for depth_bin in sorted(depth_bins, reverse=True):
            base_row = depth_bin_to_base_row[depth_bin]
            
            if depth_bin != max(depth_bins):
                plt.axhline(y=base_row, color='white', linestyle='-', alpha=0.3, linewidth=0.5)
            
            if depth_bin % 1000 == 0:
                y_tick_indices.append(base_row + slots_per_bin // 2)
                y_tick_labels.append(f'{depth_bin}')

        plt.yticks(y_tick_indices, y_tick_labels)
        plt.tight_layout()
        pdf.savefig()
        plt.close()
print(f"Saved heatmaps to {pdf_filename}")

Saved heatmaps to D:\Neural-Pipeline\results\20250710_dots3DMP_singleneuron_heatmap.pdf
